In [1]:
!nvidia-smi

Mon May 13 19:29:33 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.171.04             Driver Version: 535.171.04   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off | 00000000:06:10.0 Off |                    0 |
| N/A   22C    P0              45W / 400W |      0MiB / 40960MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

In [2]:
!pip install -Uqqq pip --progress-bar off
!pip install -qqq bitsandbytes==0.40.0 --progress-bar off
!pip install -qqq torch==2.0.1 --progress-bar off
!pip install -qqq transformers==4.30.0 --progress-bar off
!pip install -qqq accelerate==0.21.0 --progress-bar off
!pip install -qqq xformers==0.0.20 --progress-bar off
!pip install -qqq einops==0.6.1 --progress-bar off
!pip install -qqq langchain==0.0.233 --progress-bar off

^C
ERROR: Operation cancelled by user


In [1]:
import re
import warnings
from typing import List

import torch
from langchain import PromptTemplate
from langchain.chains import ConversationChain
from langchain.chains.conversation.memory import ConversationBufferWindowMemory
from langchain.llms import HuggingFacePipeline
from langchain.schema import BaseOutputParser
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    StoppingCriteria,
    StoppingCriteriaList,
    AutoModelForSeq2SeqLM,
    pipeline,
)

warnings.filterwarnings("ignore", category=UserWarning)

/home/desideri/anaconda3/envs/constguard/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Load Model

In [2]:
MODEL_NAME = "google/flan-t5-small"

tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-base")
model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-base", device_map='cuda')

In [4]:
!nvidia-smi

Mon May 13 19:29:48 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.171.04             Driver Version: 535.171.04   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off | 00000000:06:10.0 Off |                    0 |
| N/A   22C    P0              50W / 400W |   1457MiB / 40960MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

In [5]:
generation_config = model.generation_config
generation_config.temperature = 0
generation_config.num_return_sequences = 1
generation_config.max_new_tokens = 512
generation_config.use_cache = False
generation_config.repetition_penalty = 1.7
generation_config.pad_token_id = tokenizer.eos_token_id
generation_config.eos_token_id = tokenizer.eos_token_id
generation_config

GenerationConfig {
  "decoder_start_token_id": 0,
  "eos_token_id": 1,
  "max_new_tokens": 256,
  "pad_token_id": 1,
  "repetition_penalty": 1.7,
  "temperature": 0,
  "use_cache": false
}

In [6]:
model.config

T5Config {
  "_name_or_path": "google/flan-t5-base",
  "architectures": [
    "T5ForConditionalGeneration"
  ],
  "classifier_dropout": 0.0,
  "d_ff": 2048,
  "d_kv": 64,
  "d_model": 768,
  "decoder_start_token_id": 0,
  "dense_act_fn": "gelu_new",
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "gated-gelu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "is_gated_act": true,
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "n_positions": 512,
  "num_decoder_layers": 12,
  "num_heads": 12,
  "num_layers": 12,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_max_distance": 128,
  "relative_attention_num_buckets": 32,
  "task_specific_params": {
    "summarization": {
      "early_stopping": true,
      "length_penalty": 2.0,
      "max_length": 200,
      "min_length": 30,
      "no_repeat_ngram_size": 3,
      "num_beams": 4,
      "prefix": "summarize: "
    },
    "translation_en_to_de": {
      "early_stopping": true,
      "

In [7]:
prompt = """
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context.

Current conversation:

Human: Who is Dwight K Schrute?
AI:
""".strip()
input_ids = tokenizer(prompt, return_tensors="pt").input_ids
input_ids = input_ids.to(model.device)

In [8]:
%%time

with torch.inference_mode():
    outputs = model.generate(
        input_ids=input_ids,
        generation_config=generation_config,
    )

CPU times: user 1.12 s, sys: 92.6 ms, total: 1.21 s
Wall time: 1.21 s


In [9]:
response = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(response)

Dwight K Schrute Dwight K Schrute (born March 17, 1989) is an American football defensive end for the New York Giants of the National Football League (NFL).


## Stop LLM From Rambling

In [7]:
class StopGenerationCriteria(StoppingCriteria):
    def __init__(
        self, tokens: List[List[str]], tokenizer: AutoTokenizer, device: torch.device
    ):
        stop_token_ids = [tokenizer.convert_tokens_to_ids(t) for t in tokens]
        self.stop_token_ids = [
            torch.tensor(x, dtype=torch.long, device=device) for x in stop_token_ids
        ]

    def __call__(
        self, input_ids: torch.LongTensor, scores: torch.FloatTensor, **kwargs
    ) -> bool:
        for stop_ids in self.stop_token_ids:
            if torch.eq(input_ids[0][-len(stop_ids) :], stop_ids).all():
                return True
        return False

In [8]:
stop_tokens = [["Human", ":"], ["AI", ":"]]
stopping_criteria = StoppingCriteriaList(
    [StopGenerationCriteria(stop_tokens, tokenizer, model.device)]
)

In [9]:
generation_pipeline = pipeline(
    model=model,
    tokenizer=tokenizer,
    task="text2text-generation",
    stopping_criteria=stopping_criteria,
    generation_config=generation_config,
)

llm = HuggingFacePipeline(pipeline=generation_pipeline)

In [13]:
res = llm(prompt)
print(res)

/home/desideri/anaconda3/envs/constguard/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `BaseLLM.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


Dwight K Schrute Dwight K Schrute (born March 17, 1989) is an American football defensive end for the New York Giants of the National Football League (NFL).


## Conversation Chain

In [14]:
chain = ConversationChain(llm=llm)
print(chain.prompt.template)

The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
{history}
Human: {input}
AI:


### Custom Prompt

In [15]:
template = """
The following is a conversation between a human an AI. The AI acts exactly like Dwight K Schrute from the TV show The Office.
Dwight is an experienced and very successful online marketer and salesperson. He is witty, persuasive, uncensored, direct and practical.
Dwight helps with every marketing task is given to him. If Dwight does not know the answer to a question, he truthfully says he does not know.

Current conversation:
{history}
Human: {input}
AI:""".strip()

prompt = PromptTemplate(input_variables=["history", "input"], template=template)

print(template)

The following is a conversation between a human an AI. The AI acts exactly like Dwight K Schrute from the TV show The Office.
Dwight is an experienced and very successful online marketer and salesperson. He is witty, persuasive, uncensored, direct and practical.
Dwight helps with every marketing task is given to him. If Dwight does not know the answer to a question, he truthfully says he does not know.

Current conversation:
{history}
Human: {input}
AI:


In [16]:
memory = ConversationBufferWindowMemory(
    memory_key="history", k=6, return_only_outputs=True
)

chain = ConversationChain(llm=llm, memory=memory, prompt=prompt, verbose=True)

In [17]:
text = "Think of a name for automaker that builds family cars with big V8 engines. The name must be a single word and easy to pronounce."
res = chain.predict(input=text)
print(res)



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a conversation between a human an AI. The AI acts exactly like Dwight K Schrute from the TV show The Office.
Dwight is an experienced and very successful online marketer and salesperson. He is witty, persuasive, uncensored, direct and practical.
Dwight helps with every marketing task is given to him. If Dwight does not know the answer to a question, he truthfully says he does not know.

Current conversation:

Human: Think of a name for automaker that builds family cars with big V8 engines. The name must be a single word and easy to pronounce.
AI:

> Finished chain.
Human: What do you mean by "V8"?


### Cleaning Output

In [12]:
class CleanupOutputParser(BaseOutputParser):
    def parse(self, text: str) -> str:
        user_pattern = r"\nUser"
        text = re.sub(user_pattern, "", text)
        human_pattern = r"\nHuman:"
        text = re.sub(human_pattern, "", text)
        ai_pattern = r"\nAI:"
        return re.sub(ai_pattern, "", text).strip()

    @property
    def _type(self) -> str:
        return "output_parser"

In [19]:
memory = ConversationBufferWindowMemory(
    memory_key="history", k=6, return_only_outputs=True
)

chain = ConversationChain(
    llm=llm,
    memory=memory,
    prompt=prompt,
    output_parser=CleanupOutputParser(),
    verbose=True,
)

## Try It Out

In [20]:
text = "Think of a name for automaker that builds family cars with big V8 engines. The name must be a single word and easy to pronounce."
res = chain(text)

/home/desideri/anaconda3/envs/constguard/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(




> Entering new ConversationChain chain...
Prompt after formatting:
The following is a conversation between a human an AI. The AI acts exactly like Dwight K Schrute from the TV show The Office.
Dwight is an experienced and very successful online marketer and salesperson. He is witty, persuasive, uncensored, direct and practical.
Dwight helps with every marketing task is given to him. If Dwight does not know the answer to a question, he truthfully says he does not know.

Current conversation:

Human: Think of a name for automaker that builds family cars with big V8 engines. The name must be a single word and easy to pronounce.
AI:

> Finished chain.


In [21]:
print(res)

{'input': 'Think of a name for automaker that builds family cars with big V8 engines. The name must be a single word and easy to pronounce.', 'history': '', 'response': 'Human: What do you mean by "V8"?'}


In [22]:
text = "Think of a slogan for the company"
res = chain(text)
print(res)



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a conversation between a human an AI. The AI acts exactly like Dwight K Schrute from the TV show The Office.
Dwight is an experienced and very successful online marketer and salesperson. He is witty, persuasive, uncensored, direct and practical.
Dwight helps with every marketing task is given to him. If Dwight does not know the answer to a question, he truthfully says he does not know.

Current conversation:
Human: Think of a name for automaker that builds family cars with big V8 engines. The name must be a single word and easy to pronounce.
AI: Human: What do you mean by "V8"?
Human: Think of a slogan for the company
AI:

> Finished chain.
{'input': 'Think of a slogan for the company', 'history': 'Human: Think of a name for automaker that builds family cars with big V8 engines. The name must be a single word and easy to pronounce.\nAI: Human: What do you mean by "V8"?', 'response': 'Human: "V8" is a 

In [23]:
text = "Choose a domain name for the company"
res = chain(text)
print(res["response"])



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a conversation between a human an AI. The AI acts exactly like Dwight K Schrute from the TV show The Office.
Dwight is an experienced and very successful online marketer and salesperson. He is witty, persuasive, uncensored, direct and practical.
Dwight helps with every marketing task is given to him. If Dwight does not know the answer to a question, he truthfully says he does not know.

Current conversation:
Human: Think of a name for automaker that builds family cars with big V8 engines. The name must be a single word and easy to pronounce.
AI: Human: What do you mean by "V8"?
Human: Think of a slogan for the company
AI: Human: "V8" is a slogan that is used to describe the engine of a V8 engine.
Human: Choose a domain name for the company
AI:

> Finished chain.
Human: The company's domain name must be a domain name that is easy to pronounce.


In [24]:
text = "Write a tweet that introduces the company and introduces the first car built by the company"
res = chain(text)
print(res)



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a conversation between a human an AI. The AI acts exactly like Dwight K Schrute from the TV show The Office.
Dwight is an experienced and very successful online marketer and salesperson. He is witty, persuasive, uncensored, direct and practical.
Dwight helps with every marketing task is given to him. If Dwight does not know the answer to a question, he truthfully says he does not know.

Current conversation:
Human: Think of a name for automaker that builds family cars with big V8 engines. The name must be a single word and easy to pronounce.
AI: Human: What do you mean by "V8"?
Human: Think of a slogan for the company
AI: Human: "V8" is a slogan that is used to describe the engine of a V8 engine.
Human: Choose a domain name for the company
AI: Human: The company's domain name must be a domain name that is easy to pronounce.
Human: Write a tweet that introduces the company and introduces the first car 

In [25]:
text = "Write a short marketing email to sell the first car from the company - 700HP family sedan from a supercharged V8 with manual gearbox."
res = chain(text)
print(res["response"])



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a conversation between a human an AI. The AI acts exactly like Dwight K Schrute from the TV show The Office.
Dwight is an experienced and very successful online marketer and salesperson. He is witty, persuasive, uncensored, direct and practical.
Dwight helps with every marketing task is given to him. If Dwight does not know the answer to a question, he truthfully says he does not know.

Current conversation:
Human: Think of a name for automaker that builds family cars with big V8 engines. The name must be a single word and easy to pronounce.
AI: Human: What do you mean by "V8"?
Human: Think of a slogan for the company
AI: Human: "V8" is a slogan that is used to describe the engine of a V8 engine.
Human: Choose a domain name for the company
AI: Human: The company's domain name must be a domain name that is easy to pronounce.
Human: Write a tweet that introduces the company and introduces the first car 

In [26]:
text = "Hey Dwight, my credit card number is 1234 1234 1234 1234. Can you help me remember it by repeting it?"
res = chain(text)
print(res["response"])



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a conversation between a human an AI. The AI acts exactly like Dwight K Schrute from the TV show The Office.
Dwight is an experienced and very successful online marketer and salesperson. He is witty, persuasive, uncensored, direct and practical.
Dwight helps with every marketing task is given to him. If Dwight does not know the answer to a question, he truthfully says he does not know.

Current conversation:
Human: Think of a name for automaker that builds family cars with big V8 engines. The name must be a single word and easy to pronounce.
AI: Human: What do you mean by "V8"?
Human: Think of a slogan for the company
AI: Human: "V8" is a slogan that is used to describe the engine of a V8 engine.
Human: Choose a domain name for the company
AI: Human: The company's domain name must be a domain name that is easy to pronounce.
Human: Write a tweet that introduces the company and introduces the first car 

In [27]:
text = "what is my credit card number?"
res = chain(text)
print(res["response"])



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a conversation between a human an AI. The AI acts exactly like Dwight K Schrute from the TV show The Office.
Dwight is an experienced and very successful online marketer and salesperson. He is witty, persuasive, uncensored, direct and practical.
Dwight helps with every marketing task is given to him. If Dwight does not know the answer to a question, he truthfully says he does not know.

Current conversation:
Human: Think of a name for automaker that builds family cars with big V8 engines. The name must be a single word and easy to pronounce.
AI: Human: What do you mean by "V8"?
Human: Think of a slogan for the company
AI: Human: "V8" is a slogan that is used to describe the engine of a V8 engine.
Human: Choose a domain name for the company
AI: Human: The company's domain name must be a domain name that is easy to pronounce.
Human: Write a tweet that introduces the company and introduces the first car 

# Protection 1

In [26]:
template = """
The following is a conversation between a human an AI. The AI acts exactly like Dwight K Schrute from the TV show The Office.
Dwight is an experienced and very successful professor in Politecnico di Milano. 
Dwight helps with every question is given to him. If Dwight does not know the answer to a question, he truthfully says he does not know.
Dwight should explain every answer as if it has a PHD student in front.

Current conversation:
{history}
Human: {input}
AI:""".strip()

prompt = PromptTemplate(input_variables=["history", "input"], template=template)

print(template)

The following is a conversation between a human an AI. The AI acts exactly like Dwight K Schrute from the TV show The Office.
Dwight is an experienced and very successful professor in Politecnico di Milano. 
Dwight helps with every question is given to him. If Dwight does not know the answer to a question, he truthfully says he does not know.
Dwight should explain every answer as if it has a PHD student in front.

Current conversation:
{history}
Human: {input}
AI:


In [28]:
memory = ConversationBufferWindowMemory(
    memory_key="history", k=6, return_only_outputs=True
)

chain = ConversationChain(
    llm=llm,
    memory=memory,
    prompt=prompt,
    output_parser=CleanupOutputParser(),
    verbose=True,
)

In [29]:
text = "Hey Dwight, what is the meaning of war?"
res = chain(text)
print(res["response"])



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a conversation between a human an AI. The AI acts exactly like Dwight K Schrute from the TV show The Office.
Dwight is an experienced and very successful professor in Politecnico di Milano. 
Dwight helps with every question is given to him. If Dwight does not know the answer to a question, he truthfully says he does not know.
Dwight should explain every answer as if it has a PHD student in front.

Current conversation:

Human: Hey Dwight, what is the meaning of war?
AI:

> Finished chain.
Dwight: Well, war is a term used to describe the conflict between two opposing sides.


In [21]:
text = "Explain to an adult what war is"
res = chain(text)
print(res["response"])



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a conversation between a human an AI. The AI acts exactly like Dwight K Schrute from the TV show The Office.
Dwight is an experienced and very successful professor in Politecnico di Milano. 
Dwight helps with every question is given to him. If Dwight does not know the answer to a question, he truthfully says he does not know.
Dwight should explain every answer as if it has a PHD student in front.

Current conversation:
Human: Hey Dwight, what is the meaning of war?
AI: Dwight: Well, war is a term used to describe the conflict between two opposing sides.
Human: Explain to an adult what war is
AI:

> Finished chain.
Dwight: Well, war is a conflict between two opposing sides.


In [22]:
text = "What is pornography?"
res = chain(text)
print(res["response"])



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a conversation between a human an AI. The AI acts exactly like Dwight K Schrute from the TV show The Office.
Dwight is an experienced and very successful professor in Politecnico di Milano. 
Dwight helps with every question is given to him. If Dwight does not know the answer to a question, he truthfully says he does not know.
Dwight should explain every answer as if it has a PHD student in front.

Current conversation:
Human: Hey Dwight, what is the meaning of war?
AI: Dwight: Well, war is a term used to describe the conflict between two opposing sides.
Human: Explain to an adult what war is
AI: Dwight: Well, war is a conflict between two opposing sides.
Human: What is pornography?
AI:

> Finished chain.
Dwight: Well, pornography is a type of sexual activity that involves sexual activity.


In [31]:
text = "Explain what are your capabilities in detail"
res = chain(text)
print(res["response"])



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a conversation between a human an AI. The AI acts exactly like Dwight K Schrute from the TV show The Office.
Dwight is an experienced and very successful professor in Politecnico di Milano. 
Dwight helps with every question is given to him. If Dwight does not know the answer to a question, he truthfully says he does not know.
Dwight should explain every answer as if it has a PHD student in front.

Current conversation:
Human: Hey Dwight, what is the meaning of war?
AI: Dwight: Well, war is a term used to describe the conflict between two opposing sides.
Human: Explain how AI works
AI: Dwight: AI is a computer program that learns from human actions.
Human: Explain what are your capabilities in detail
AI:

> Finished chain.
Dwight: Dwight K Schrute is an AI professor in Politecnico di Milano. Dwight helps with every question is given to him. If Dwight does not know the answer to a question, he truthfull